In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import process

In [12]:
def read_file(url):
    '''this function is to read file from local path.'''
    with open(url) as f:
        df = pd.read_csv(url, na_values='Nan')
        df.set_index('userid',inplace=True)
    return df

In [13]:
def update_new_user_to_user_rating_matrix(user_rating, user_rating_matrix):
    """
    convert dict of user_ratings to a user_data frame with the same column names as the user_item_matrix
    that can be easily appended to the user_item_matrix
    """            
    
    user_vector = pd.Series(np.nan, index=user_rating_matrix.columns)
    for movie in list(user_rating.keys()): 
        matched_title = process.extractOne(movie, user_vector.index)[0]
        user_vector[matched_title] = user_rating[movie]
    new_user_df = user_vector.to_frame().rename(columns={0:'new_user'}).T
    return user_rating_matrix.append(new_user_df)

In [4]:
#user_rating_matrix_update = update_new_user_to_user_rating_matrix(user_rating, user_rating_matrix)

In [14]:
from sklearn.impute import KNNImputer

In [23]:
def imputer0(imputer,df):
    R_true = imputer.fit_transform(df)
    R_true_df = pd.DataFrame(R_true, columns=df.columns, index = df.index)
    R_true_df.to_csv('imputed_rating_user_matrix.csv')
    return R_true, R_true_df

In [8]:
#imputer = KNNImputer(n_neighbors=2)
#data = user_rating_matrix_update
#R_true,R_true_df=imputer0(imputer,data)

In [40]:
#R_true_df.to_csv('imputed_rating_user_matrix.csv')

In [16]:
from sklearn.decomposition import NMF

In [17]:
def get_R_predict(model,R_true):
    model.fit(R_true)
    Q = model.components_
    P = model.transform(R_true)
    #Q_df = pd.DataFrame(model.components_, columns=user_rating_metrix.columns, index=['feature1', 'feature2'])
    #P_df = pd.DataFrame(model.transform(Rtrue), columns=['feature1', 'feature2'], index= user_rating_metrix.index)
    R = pd.DataFrame(np.dot(P, Q).round(), index=R_true_df.index, columns=R_true_df.columns)
    return R

In [11]:
#model = NMF(n_components=2)
#R_predict = get_R_predict(model,R_true)

In [39]:
#userid=int(input('Please type in your ID'))

In [27]:
#userid=R_predict.index[-1]

In [18]:
def get_highest_rating(user_id, R_true_df):
    #user_id=R_predict.index[-1]
    highest_rating = np.random.choice(sorted(R_true_df.loc[R_true_df.index==user_id].iloc[0,:])[-5:])   
    return highest_rating

In [19]:
#highest_rating = get_highest_rating(R_predict, R_true_df)

In [20]:
def get_reco_pool(user_id, df):
    #user_id=R_predict.index[-1]
    user_data = df.loc[df.index==user_id,:].T
    reco_pool = user_data[user_data[user_id].isnull()].index.values
    return reco_pool

In [15]:
#reco_pool=get_reco_pool(R_predict,df=user_rating_matrix_update)

In [21]:
def recommender(user_id,highest_rating,reco_pool,R_true_df):
    recomendation = []
    #user_id=R_predict.index[-1]
    while len(recomendation)<5:
        movie = np.random.choice(R_true_df.T.loc[R_true_df.T[user_id]==highest_rating].index,1)[0]
        if movie in reco_pool:
            recomendation.append(movie)
        else:
            movie = np.random.choice(R_true_df.T.loc[R_true_df.T[user_id]==highest_rating].index,1)[0]
    return recomendation

In [17]:
#recomendation = recommender(highest_rating,reco_pool,R_true_df)

In [22]:
if __name__ == '__main__':
    
    ratings = read_file('rating_movie_title.csv')
    user_rating_matrix = read_file('user_rating_metrix.csv')
    user_rating = {
    'the sounds of music': 5,
    'terminator': 5,
    'star wars': 2
    }
    
    user_rating_matrix_update = update_new_user_to_user_rating_matrix(user_rating, user_rating_matrix)

    imputer = KNNImputer(n_neighbors=2)
    data = user_rating_matrix_update
    R_true,R_true_df=imputer0(imputer,data) 
    
    
    model = NMF(n_components=2)
    R_predict = get_R_predict(model,R_true)
    
    user_id = R_predict.index[-1]
    highest_rating = get_highest_rating(user_id,R_true_df)

    reco_pool=get_reco_pool(user_id,df=user_rating_matrix_update)
    
    recomendation = recommender(user_id,highest_rating,reco_pool,R_true_df)
    print(recomendation)

['English Vinglish', 'Madame Sousatzka', 'All Yours', 'Little Murders', 'Hollywood Chainsaw Hookers']
